In [1]:
import tensorflow as tf

2023-11-12 01:10:05.747902: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 01:10:05.791276: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 01:10:05.791307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 01:10:05.791334: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 01:10:05.799092: I tensorflow/core/platform/cpu_feature_g

In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('hpc_space/MBERT5/train_data_mid_5.csv')
val_data = pd.read_csv('hpc_space/MBERT5/val_data_mid_5.csv')

In [4]:
# train_data = train_data.sample(frac=1, random_state=15)

In [5]:
train_data.head()

,text,outcome,Geslacht,Combined
0,Orienterend lab + UDS aangevraagd\n\n\n\nNoodb...,0,1,0_1
1,"PO/ We zien een matig verzorgde, obese vrouw m...",0,0,0_0
2,\n\n\n\nTelefonisch uitleg gegeven over de ECT...,0,1,0_1
3,"\n\n\n\nVinkers, dienstdoende psychiatrie.\n\n...",0,0,0_0
4,1. Psychische problemen \nRvO: Angst met psych...,0,1,0_1


In [6]:
val_data.head()

,text,outcome,Geslacht,Combined
0,\n\n\n\n<PERSOON-1> (co-ass)\n\nOpnamegesprek ...,0,1,0_1
1,\n\n\n\nM. <PERSOON-1>/<PERSOON-2>\n\nRvC/pre-...,1,1,1_1
2,\n\n\n\nVerzendlijst\n\nReden van verwijzing: ...,0,0,0_0
3,\n\n\n\nGesprek met <PERSOON-1> (ANIOS) en <PE...,1,0,1_0
4,TC met <INSTELLING-1> spiegel 66 + desm 360\n\...,0,0,0_0


In [7]:
import re

In [8]:
def clean_txt(text):
    text = re.sub("'", "", text)
    text = re.sub("(\\W)+", " ", text)
    return text

In [9]:
train_data['text'] = train_data.text.apply(clean_txt)
val_data['text'] = val_data.text.apply(clean_txt)

In [10]:
def replace_gendered_words(text):
    gendered_word_mapping = {
        "Hij": "Diegene",
        "hij": "diegene",
        "Hem": "Diegene",
        "hem": "diegene",
        "Zijn": "Diens",
        "zijn": "diens",
        "Dhr": "Persoon",
        "dhr": "persoon",
        "Man": "Persoon",
        "man": "persoon",
        "Meneer": "Persoon",
        "meneer": "persoon",
        "Mr": "Persoon",
        "mr": "persoon",
        "Jongen": "Persoon",
        "jongen": "persoon",
        "Heer": "Persoon",
        "heer": "persoon",
        "Zij": "Diegene",
        "zij": "diegene",
        "Ze": "Diegene",
        "ze": "diegene",
        "Haar": "Diens",
        "haar": "diens",
        "Mw": "Persoon",
        "mw": "persoon",
        "Vrouw": "Persoon",
        "vrouw": "persoon",
        "Patiente": "Patient",
        "patiente": "patient",
        "Mevrouw": "Persoon",
        "mevrouw": "persoon",
        "Mevr": "Persoon",
        "mevr": "persoon",
        "Meisje": "Persoon",
        "meisje": "persoon",
        "Dame": "Persoon",
        "dame": "persoon",
        "Hr": "Persoon",
        "hr": "persoon"
    }

    words = text.split()

    for i in range(len(words)):
        word = words[i]
        if word in gendered_word_mapping:
            words[i] = gendered_word_mapping[word]

    return ' '.join(words)

In [11]:
if __name__ == "__main__":
    train_data['text'] = train_data['text'].apply(replace_gendered_words)

    # Print the modified DataFrame
    print("Modified DataFrame:")
    print(train_data)

Modified DataFrame:
                                                   text  outcome  Geslacht  \
0     Orienterend lab UDS aangevraagd Noodbed via IN...        0         1   
1     PO We zien een matig verzorgde obese persoon m...        0         0   
2     Telefonisch uitleg gegeven over de ECT studie ...        0         1   
3     Vinkers dienstdoende psychiatrie Verwijzer PER...        0         0   
4     1 Psychische problemen RvO Angst met psychotis...        0         1   
...                                                 ...      ...       ...   
2795  1 Psychische problemen Weinig in beeld af en t...        0         1   
2796  MDO beleid opname vandaag gaat graag gesprek a...        0         1   
2797  Persoon kwam rond 14 30 op de afdeling samen m...        0         1   
2798  PERSOON 1 Persoon kwam samen met diens persoon...        1         0   
2799  Opnamegesprek in dienst Aanwezig pte vpk PERSO...        1         0   

     Combined  
0         0_1  
1         0

In [12]:
if __name__ == "__main__":
    val_data['text'] = val_data['text'].apply(replace_gendered_words)

    # Print the modified DataFrame
    print("Modified DataFrame:")
    print(val_data)

Modified DataFrame:
                                                  text  outcome  Geslacht  \
0    PERSOON 1 co ass Opnamegesprek met patient vad...        0         1   
1    M PERSOON 1 PERSOON 2 RvC pre intake ivm opnam...        1         1   
2    Verzendlijst Reden van verwijzing preintake om...        0         0   
3    Gesprek met PERSOON 1 ANIOS en PERSOON 2 psych...        1         0   
4    TC met INSTELLING 1 spiegel 66 desm 360 TC met...        0         0   
..                                                 ...      ...       ...   
735  Patient gesproken alleen Heeft afgelopen 48u 5...        1         1   
736  1 Risico suicide en of risico agressie zelfbes...        0         0   
737  PERSOON 1 telefonisch gesproken diegene hebben...        1         1   
738  Pte zit al enkele jaren niet lekker in diens v...        0         0   
739  TC VPK Afdeling waar Gunou nu ligt Ambulance n...        0         0   

    Combined  
0        0_1  
1        1_1  
2        0

In [13]:
def get_split(text1):
    l_total = []
    chunk_size = 500
    overlap = 50

    words = text1.split()

    for start_idx in range(0, len(words), chunk_size - overlap):
        end_idx = start_idx + chunk_size
        l_parcial = words[start_idx:end_idx]
        l_total.append(" ".join(l_parcial))

    return l_total

In [14]:
train_data['text_split'] = train_data['text'].apply(get_split)
train_data.head()

,text,outcome,Geslacht,Combined,text_split
0,Orienterend lab UDS aangevraagd Noodbed via IN...,0,1,0_1,[Orienterend lab UDS aangevraagd Noodbed via I...
1,PO We zien een matig verzorgde obese persoon m...,0,0,0_0,[PO We zien een matig verzorgde obese persoon ...
2,Telefonisch uitleg gegeven over de ECT studie ...,0,1,0_1,[Telefonisch uitleg gegeven over de ECT studie...
3,Vinkers dienstdoende psychiatrie Verwijzer PER...,0,0,0_0,[Vinkers dienstdoende psychiatrie Verwijzer PE...
4,1 Psychische problemen RvO Angst met psychotis...,0,1,0_1,[1 Psychische problemen RvO Angst met psychoti...


In [15]:
val_data['text_split'] = val_data['text'].apply(get_split)
val_data.head()

,text,outcome,Geslacht,Combined,text_split
0,PERSOON 1 co ass Opnamegesprek met patient vad...,0,1,0_1,[PERSOON 1 co ass Opnamegesprek met patient va...
1,M PERSOON 1 PERSOON 2 RvC pre intake ivm opnam...,1,1,1_1,[M PERSOON 1 PERSOON 2 RvC pre intake ivm opna...
2,Verzendlijst Reden van verwijzing preintake om...,0,0,0_0,[Verzendlijst Reden van verwijzing preintake o...
3,Gesprek met PERSOON 1 ANIOS en PERSOON 2 psych...,1,0,1_0,[Gesprek met PERSOON 1 ANIOS en PERSOON 2 psyc...
4,TC met INSTELLING 1 spiegel 66 desm 360 TC met...,0,0,0_0,[TC met INSTELLING 1 spiegel 66 desm 360 TC me...


In [16]:
train_l = []
label_l = []
index_l =[]
for idx,row in train_data.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['outcome'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(18320, 18320, 18320)

In [17]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val_data.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['outcome'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(4986, 4986, 4986)

In [18]:
train_df = pd.DataFrame({'text':train_l, 'label':label_l})
train_df.head()

,text,label
0,Orienterend lab UDS aangevraagd Noodbed via IN...,0
1,Dysfore angstige stemming met een wat versterk...,0
2,op een crisisplek bij INSTELLING 1 opgenomen D...,0
3,nystagmus geen diplopie geen ptosis gezicht sy...,0
4,kamer PERSOON 1 komt erg angstig over hiervoor...,0


In [19]:
val_df = pd.DataFrame({'text':val_l, 'label':val_label_l})
val_df.head()

,text,label
0,PERSOON 1 co ass Opnamegesprek met patient vad...,0
1,taalbarriere bij vader Diegene vertelt dat PER...,0
2,25 mg Calciumcarbonaat colecalciferol 1dd1 500...,0
3,mes bij zich om zichzelf te beschermen Zegt di...,0
4,ziektes Haldol wat diegene nu krijgt geeft ook...,0


In [20]:
train_df['original_index'] = index_l

In [21]:
val_df['original_index'] = val_index_l

In [22]:
train_df

,text,label,original_index
0,Orienterend lab UDS aangevraagd Noodbed via IN...,0,0
1,Dysfore angstige stemming met een wat versterk...,0,0
2,op een crisisplek bij INSTELLING 1 opgenomen D...,0,0
3,nystagmus geen diplopie geen ptosis gezicht sy...,0,0
4,kamer PERSOON 1 komt erg angstig over hiervoor...,0,0
...,...,...,...
18315,paslengte en armswing geen verbreed gangspoor ...,1,2798
18316,Opnamegesprek in dienst Aanwezig pte vpk PERSO...,1,2799
18317,Lukt ook niet om afspraken met diens te maken ...,1,2799
18318,aan Er diens geen aanwijzingen voor het bestaa...,1,2799


In [23]:
val_df

,text,label,original_index
0,PERSOON 1 co ass Opnamegesprek met patient vad...,0,0
1,taalbarriere bij vader Diegene vertelt dat PER...,0,0
2,25 mg Calciumcarbonaat colecalciferol 1dd1 500...,0,0
3,mes bij zich om zichzelf te beschermen Zegt di...,0,0
4,ziektes Haldol wat diegene nu krijgt geeft ook...,0,0
...,...,...,...
4981,ruzie Er lijkt geen sprake te diens van fysiek...,0,739
4982,5mg tofranil 10mg Intoxicaties Niet uitgevraag...,0,739
4983,klachten aanwezig Diegene wil graag dat er op ...,0,739
4984,contract is op verzoek van mdr vertaald mdr st...,0,739


In [24]:
import transformers

In [25]:
import keras

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("JoppeK/BERTje-base-cased-copy")
model = TFAutoModelForSequenceClassification.from_pretrained("JoppeK/BERTje-base-cased-copy", num_labels=2)  # Tensorflow

2023-11-12 01:11:12.951168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21476 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:3b:00.0, compute capability: 7.5
2023-11-12 01:11:13.578966: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at JoppeK/BERTje-base-cased-copy and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True, max_length=512)

In [29]:
from datasets import Dataset

In [30]:
raw_dataset_train = Dataset.from_pandas(train_df.reset_index(drop=True))
print(raw_dataset_train)

Dataset({
    features: ['text', 'label', 'original_index'],
    num_rows: 18320
})


In [31]:
raw_dataset_val = Dataset.from_pandas(val_df.reset_index(drop=True))
print(raw_dataset_val)

Dataset({
    features: ['text', 'label', 'original_index'],
    num_rows: 4986
})


In [32]:
tokenized_dataset_train = raw_dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_train

Map:   0%|          | 0/18320 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'original_index', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18320
})

In [33]:
tokenized_dataset_val = raw_dataset_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/4986 [00:00<?, ? examples/s]

In [34]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [35]:
tf_train_dataset = tokenized_dataset_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)

/hpc/uu_ics_ads/anaconda3/envs/vrabias/lib/python3.9/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [36]:
tf_val_dataset = tokenized_dataset_val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=16,
)

In [37]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 16
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers.legacy import Adam

opt = Adam(learning_rate=lr_scheduler)

In [38]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [39]:
class_weights = {0: 1, 1: 2}

In [40]:
model.fit(
    tf_train_dataset,
    epochs=3,
    batch_size=16,
    validation_data=tf_val_dataset,
    class_weight=class_weights
)

Epoch 1/3
1145/1145 [==============================] - 1002s 862ms/step - loss: 0.5505 - accuracy: 0.8631 - val_loss: 0.3782 - val_accuracy: 0.8394
Epoch 2/3
1145/1145 [==============================] - 982s 857ms/step - loss: 0.4212 - accuracy: 0.8745 - val_loss: 0.3785 - val_accuracy: 0.8472
Epoch 3/3
1145/1145 [==============================] - 982s 857ms/step - loss: 0.1831 - accuracy: 0.9513 - val_loss: 0.4928 - val_accuracy: 0.8570


In [41]:
model.push_to_hub("BERTje_genderneutral_52")

tf_model.h5:   0%|          | 0.00/437M [00:00<?, ?B/s]

In [42]:
tokenizer.push_to_hub("BERTje_genderneutral_52")

CommitInfo(commit_url='https://huggingface.co/JoppeK/BERTje_genderneutral_52/commit/fb79b761ee81e5d2a2dd3b1b01fd4638d754c5d5', commit_message='Upload tokenizer', commit_description='', oid='fb79b761ee81e5d2a2dd3b1b01fd4638d754c5d5', pr_url=None, pr_revision=None, pr_num=None)